In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d angelolmg/tilda-400-64x64-patches

In [ ]:
!unzip '/content/tilda-400-64x64-patches.zip' -d '/content/dataset'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os
x = os.listdir('/content/dataset/thread-error')
len(x)

In [ ]:
import os
import shutil

def copy_images(source_dir, target_dir, num_images_from_good):
    # Create target directory if it doesn't exist
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Copy images from other directories
    for class_name in os.listdir(source_dir):
        if class_name == 'good':
            continue  # Skip 'good' directory for now
        class_dir = os.path.join(source_dir, class_name)
        target_class_dir = os.path.join(target_dir, class_name)
        shutil.copytree(class_dir, target_class_dir)

    # Copy only 800 images from the 'good' directory
    good_dir = os.path.join(source_dir, 'good')
    target_good_dir = os.path.join(target_dir, 'good')
    os.makedirs(target_good_dir)
    good_images = os.listdir(good_dir)[:num_images_from_good]
    for img in good_images:
        img_path = os.path.join(good_dir, img)
        shutil.copy(img_path, target_good_dir)

# Define source and target directories
source_directory = '/content/dataset'
target_directory = '/content/data'
num_images_from_good = 800

# Copy images to target directory
copy_images(source_directory, target_directory, num_images_from_good)

In [ ]:
import os 
d = os.listdir('/content/data/good')
len(d)

In [ ]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    zoom_range = 0.3,
    shear_range = 0.2,
    vertical_flip = 'True',
    horizontal_flip = 'True'
)

In [ ]:
train_gen = datagen.flow_from_directory(
    '/content/data',
    target_size = (64,64),
    classes = ['good', 'hole', 'objects', 'oil-spot', 'thread-error']
)

In [ ]:
train_gen.labels

In [ ]:
train_gen.num_classes

In [ ]:
train_gen.class_mode

In [ ]:
train_gen.image_shape

In [ ]:
conv = VGG19(
    input_shape = (64,64,3),
    include_top = False
)

In [ ]:
conv.summary()

In [ ]:
conv.trainable = False

In [ ]:
conv.summary()

In [ ]:
model_without_fine_tuning = Sequential()
model_without_fine_tuning.add(conv)
model_without_fine_tuning.add(Flatten())
model_without_fine_tuning.add(Dense(256,activation='relu'))
model_without_fine_tuning.add(Dense(5,activation='softmax'))

In [ ]:
model_without_fine_tuning.summary()

In [ ]:
model_without_fine_tuning.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
history = model_without_fine_tuning.fit_generator(train_gen,epochs=10)